In [1]:
### functional transcription


# from IPython.display import display, HTML
import sounddevice as sd
import numpy as np
import queue
from scipy.io.wavfile import write
from faster_whisper import WhisperModel
from ipywidgets import widgets, Layout, Button, VBox, Textarea, HBox, HTML
from IPython.display import display, Markdown

# Initialisieren des Whisper-Modells
# model = WhisperModel("tiny", device="cuda")
model = WhisperModel("tiny", device="cpu")

# Audioaufnahme-Einstellungen
sample_rate = 16000  # Sampling-Rate
audio_queue = queue.Queue()

# styling
# Basis-Layout-Optionen
base_layout = Layout(display='flex', justify_content='center', align_items='center')

# Stil-Optionen für Widgets
# button_layout = Layout(width='150px', height='40px', margin='5px 10px 5px 10px')
# button_layout = Layout(flex='0 1 auto', height='auto', margin='10px 0 10px 0')
transcript_layout = Layout(width='60%', min_height='auto', overflow='auto', margin='10px 0 10px 0')

# Anwendung von CSS-Klassen
# CSS für individuelles Styling einschließlich Hintergrundbild
css_style = """
<style>
    .background-container { 
      background-image: url('bg.png');
      background-repeat: no-repeat;
      background-position: center center; 
      background-size: cover;
      margin: 0; 
      height: 100vh;
      width: 100vw;
      overflow: hidden;
    }

    /* Zusätzliches Styling für das VBox-Container-Widget */
    .main-box {
      justify-content: center;
      align-items: center;
      display: flex;
      flex-direction: column;
      height: 100vh; /* Setzt die Höhe des Containers auf die Höhe des Viewports */
      width: 100vw; /* Setzt die Breite des Containers auf die Breite des Viewports */
    }
    .custom-textarea { background-color: black; color: white; font-family: 'Barlow', sans-serif; border-radius: 5px;}
    .example-button { background-color: #0B3653; color: white; font-family: 'Barlow', sans-serif; border-radius: 5px;}
    .send-button { background-color: #72B957; color: white; font-family: 'Barlow', sans-serif; border-radius: 5px;}
</style>
"""
display(HTML(css_style))

# callbacks & fcns
def audio_callback(indata, frames, time, status):
    """Diese Funktion wird von sounddevice während der Audioaufnahme aufgerufen."""
    audio_queue.put(indata.copy())

stream = sd.InputStream(callback=audio_callback, samplerate=sample_rate, channels=1, dtype='int16')

def start_recording(b):
    """Startet die Audioaufnahme."""
    audio_queue.queue.clear()  # Bereinigt die Queue vor der Aufnahme
    stream.start()

def stop_recording(b):
    """Stoppt die Audioaufnahme und startet die Transkription."""
    stream.stop()
    audio_data = []
    while not audio_queue.empty():
        audio_data.append(audio_queue.get())
    if audio_data:
        audio_np = np.concatenate(audio_data, axis=0)
        write('output.wav', sample_rate, audio_np.astype(np.int16))  # Schreibt die WAV-Datei
        transcribe_audio('output.wav')

def transcribe_audio(audio_path):
    """Transkribiert das aufgenommene Audio."""
    segments, _ = model.transcribe(audio_path, language="de", word_timestamps=True)
    transcription = " ".join([segment.text for segment in segments])
    text_output.value = transcription  # Aktualisiert das Text-Widget mit der Transkription

def reset_transcription(b):
    """Setzt die Transkription zurück."""
    text_output.value = ""

# text-output-widget
text_output = widgets.Textarea(placeholder='Hier wird die Transkription angezeigt...')
text_output.add_class('custom-textarea')

# Button-Layout anpassen
button_layout = Layout(width='150px', height='40px', margin='5px 10px', align_items='center')

# Setzen Sie das Layout für jeden Button
start_btn = widgets.Button(description="Start Recording", layout=button_layout)
start_btn.add_class('example-button') 
stop_btn = widgets.Button(description="Stop Recording", layout=button_layout)
stop_btn.add_class('example-button') 
reset_btn = widgets.Button(description="Reset", layout=button_layout)
reset_btn.add_class('example-button') 

# verknüpfe fcns
start_btn.on_click(start_recording)
stop_btn.on_click(stop_recording)
reset_btn.on_click(reset_transcription)

# Korrekturen im HBox-Layout für die Button-Anordnung
buttons_box_1 = HBox([start_btn, stop_btn, reset_btn], layout=Layout(display='flex', flex_flow='row nowrap', justify_content='center', align_items='center', width='100%'))

# Anordnung der Widgets mit Hintergrundklasse aktualisieren
main_box = VBox([
    HTML(value="<b>Aufnahme:</b>", style={'margin': '10px'}),
    buttons_box_1,
    HTML(value="<b>Transkript:</b>", style={'margin': '10px'}),
    text_output,
], layout=Layout(align_items='center', justify_content='center', width='100%', flex_flow='column', display='flex'))

main_box.add_class('main-box')
main_box.add_class('background-container') 

display(main_box)
